# Model profiles using different type of source redshift information as input

In this example we model lensing profiles by giving as input either : 
- discrete source redshifts, 
- a redshift distribution function,
- the value of the mean beta parameters : 
$\langle \beta_s \rangle = \left\langle \frac{D_{LS}}{D_S}\frac{D_\infty}{D_{L,\infty}}\right\rangle$ ,
$\langle \beta_s^2 \rangle = \left\langle \left(\frac{D_{LS}}{D_S}\frac{D_\infty}{D_{L,\infty}}\right)^2 \right\rangle$

In [ ]:
import warnings
warnings.filterwarnings("ignore", message='.*(!).*')
import os
## Uncomment the following line if you want to use a specific modeling backend among 'ct' (cluster-toolkit), 'ccl' (CCL) or 'nc' (Numcosmo). Default is 'ccl'
#os.environ['CLMM_MODELING_BACKEND'] = 'nc'

In [ ]:
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams.update({'figure.figsize': (20,6)})

Make sure we know which version we're using

In [ ]:
clmm.__version__

## Import mock data module and setup the configuration 

In [ ]:
from clmm.support import mock_data as mock
from clmm import Cosmology
from clmm.z_distributions import *

Mock data generation requires a defined cosmology

In [ ]:
mock_cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

Mock data generation requires some cluster information

In [ ]:
cosmo = mock_cosmo

# cluster properties from https://arxiv.org/pdf/1611.03866.pdf
cluster_id = "SPT-CL J0000−5748"
cluster_m = 4.56e14 # M500,c
cluster_z = 0.702
cluster_ra = 0.2499 
cluster_dec = -57.8064
concentration = 5 # (arbitrary value, not from the paper)

#source redshift distribution properties
cluster_beta_s_mean = 0.466
cluster_beta_s2_mean = 0.243
ngal_density= 26. *100 # density of source galaxies per arcmin^2 # (arbitrary value, not from the paper)
model_z_distrib_dict = {'func':desc_srd, 'name':"desc_srd"}
delta_z_cut = 0.1
zsrc_min = cluster_z + delta_z_cut
zsrc_max = 3.0

# 1 - Defining the different inputs for the source redshifts

## Discrete redshifts

### Generate the mock source catalog

The CLMM mock data generation will provide, among other things, a redshift value for each background galaxy that is draw from the redshift distribution given by `model_z_distrib_dict`.

In [ ]:
np.random.seed(0)
source_catalog = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, model_z_distrib_dict['name'], delta_so=500,
                                              massdef='critical', zsrc_min=zsrc_min, zsrc_max=zsrc_max, ngal_density=ngal_density, 
                                              cluster_ra=cluster_ra, cluster_dec=cluster_dec)


## Beta parameters
From this udnerlying redshift distribution, one may directly compute the average $\langle\beta_s\rangle$ and $\langle\beta_s^2\rangle$ quantities

In [ ]:
z_inf = 1000

beta_s_mean        = clmm.utils.compute_beta_s_mean(cluster_z, z_inf, cosmo, zmax=zsrc_max, delta_z_cut=delta_z_cut, zmin=None, z_distrib_func=model_z_distrib_dict['func'])
beta_s_square_mean = clmm.utils.compute_beta_s_square_mean(cluster_z, z_inf, cosmo, zmax=zsrc_max, delta_z_cut=delta_z_cut, zmin=None, z_distrib_func=model_z_distrib_dict['func'])

print ("$<\\beta_s>$ = ", round(beta_s_mean,3), ",\n$<\\beta_s^2>$ = ", round(beta_s_square_mean,3))

## Visualisation

In [ ]:
z = np.linspace(0,zsrc_max,1000)

plt.hist(source_catalog['z'], bins=50, alpha=0.3, density=True, label='mock source redshift');
plt.axvline(zsrc_min, color='red', label='requested zmin')
plt.text(4, 0.6, '$\\langle\\beta_s\\rangle$ = '+str(round(beta_s_mean,3)) + ', $\\langle\\beta_s^2\\rangle$ = ' + str(round(beta_s_square_mean,3)))
#here we multiply by a constant for visualisation purposes
plt.plot(z,model_z_distrib_dict['func'](z)*25, linestyle='dashed',label='redshift distribution fct (arbitrary normalisation)')
plt.xlabel('$z_s$')
plt.ylabel('pdf')
plt.legend()

Here we use a zmax=3 (different from default values) to highlight the importance of specifying the zmax when computing the modeling. 

# 2 - Compute models

Here we are computing the models for the tangential shear, reduced shear, convergence, magnification and magnification bias.

## Profile from mock data

First we compute the cluster profile based on the mock source catalog produced in the previous section.

In [ ]:
gc_object = clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec, 
                               cluster_z, source_catalog)

In [ ]:
gc_object.compute_tangential_and_cross_components()

In [ ]:
gc_object.make_radial_profile('Mpc', bins=10, cosmo=cosmo)

## Different ways of modeling the reduced shear

In [ ]:
#define radius
rr = np.logspace(np.log10(0.2), np.log10(5), 10)

### **Case 1 : Discrete redshift and exact formula**

#### In case we know the discrete source redshift, we can compute the reduced shear for each source galaxy and take the average at a given radius. 
#### This may take a bit of time, depending on the size of source redshift catalog and number of radius points.

In [ ]:
%%time
#in case we know the discrete source redshift, we can compute the reduced shear for each source galaxy and take the average at a given radius.
gt_discrete = np.zeros(rr.size)

for i in range(rr.size):
    gt_discrete[i] = np.mean(clmm.theory.compute_reduced_tangential_shear(rr[i], cluster_m, concentration, cluster_z, 
                                            source_catalog['z'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='discrete', approx=None))

### **Case 2 : Redshift distribution and exact formula**

#### If we don't know the exact source redshift but we know the source mean redshift distribution function, we can give it as input. \\
#### In this case, we are integrating over the distribution function so it may be quite slow.

In [ ]:
%%time
gt_distribution_no_approx = clmm.theory.compute_reduced_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', beta_kwargs={'zmax':zsrc_max}, approx=None)

### **Cases 3 : Redshift distribution and approximation**

#### If we want a faster approach we can use an approximation for the reduced shear, using 1 or 2 order of Taylor expansion for the expression of the reduced shear.

In [ ]:
%%time
gt_distribution_order1 = clmm.theory.compute_reduced_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', beta_kwargs={'zmax':zsrc_max},  approx="order1")

In [ ]:
%%time
gt_distribution_order2 = clmm.theory.compute_reduced_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', beta_kwargs={'zmax':zsrc_max},  approx="order2")

### **Cases 4 : Mean lensing efficiencies and approximation**

Finally, we can also model the reduced shear if the only information we have about the source redshift distribution is through the mean lensing efficiency parameters $\langle\beta_s\rangle$ and $\langle\beta_s^2\rangle$. \
In this case, we need to use an approximation for the formula. This is the fastest approach.

In [ ]:
%%time
gt_beta_1 = clmm.theory.compute_reduced_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', beta_kwargs={'zmax':zsrc_max}, z_src_info='beta', approx="order1")

In [ ]:
%%time
gt_beta_2 = clmm.theory.compute_reduced_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', beta_kwargs={'zmax':zsrc_max}, z_src_info='beta', approx="order2")

# 3 - Comparison of the four cases for the reduced tangnetial shear

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True)

ax1.loglog(gc_object.profile['radius'], gc_object.profile['gt'], 'gs-', label = 'data')

ax1.loglog(rr, gt_discrete, 'k.-', label = 'discrete')
ax1.loglog(rr, gt_distribution_no_approx, 'r.-', label = 'distribution, no approx')
ax1.loglog(rr, gt_distribution_order1, 'ro-', label = 'distribution, order 1 approx')
ax1.loglog(rr, gt_distribution_order2, 'rd-', label = 'distribution, order 2 approx')
ax1.loglog(rr, gt_beta_1, 'b--', label = 'beta, order 1 approx')
ax1.loglog(rr, gt_beta_2, 'bx-', label = 'beta, order 2 approx')

ax1.set_ylabel('$g_t$')
ax1.set_xlabel('radius [Mpc]')

ax1.legend()

ax2.plot(rr, 100*(gt_discrete-gt_discrete)/gt_discrete, 'k.-', label = 'discrete')
ax2.plot(rr, 100*(gt_distribution_no_approx-gt_discrete)/gt_discrete, 'r-.', label = 'distribution, no approx')
ax2.plot(rr, 100*(gt_distribution_order1-gt_discrete)/gt_discrete, 'ro-', label = 'distribution, order 1 approx')
ax2.plot(rr, 100*(gt_distribution_order2-gt_discrete)/gt_discrete, 'rd-', label = 'distribution, order 2 approx')
ax2.plot(rr, 100*(gt_beta_1-gt_discrete)/gt_discrete, 'b--', label = 'beta, order 1 approx')
ax2.plot(rr, 100*(gt_beta_2-gt_discrete)/gt_discrete, 'bx-', label = 'beta, order 2 approx')

ax2.set_ylabel('%')
ax2.set_xlabel('radius [Mpc]')

ax2.legend()


All modeled profiles give similar results. They do not correspond to the profile computed from the data in the inner part, because of different ways of constructing the profiles (taking the average radial point and reduced shear value in a bin or computing the average expected reduced shear at a given radius).

The profiles computed using an approximation for the reduced shear formula are lower by a few percents, especially in the inner region. The profiles computed from a redshift distribution or a known source redshift differ at the subpercent level.

# 4- Modeling and plotting the other quantities

Here we will just compute models for cases 1, 2 and 4. For the shear and convergence there is no need for an approximated formula. 

In [ ]:
gammat_discrete = np.zeros(rr.size)

for i in range(rr.size):
    gammat_discrete[i] = np.mean(clmm.theory.compute_tangential_shear(rr[i], cluster_m, concentration, cluster_z, 
                                            source_catalog['z'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='discrete'))
    
gammat_distribution = clmm.theory.compute_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', beta_kwargs={'zmax':zsrc_max})
    
gammat_beta = clmm.theory.compute_tangential_shear(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta', beta_kwargs={'zmax':zsrc_max})

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True)

ax1.loglog(rr, gammat_discrete, 'k.-', label = 'discrete')
ax1.loglog(rr, gammat_distribution, 'rx-', label = 'distribution, no approx')
ax1.loglog(rr, gammat_beta, 'b--', label = 'beta, no approx')

ax1.set_ylabel('$\\gamma_t$')
ax1.set_xlabel('radius [Mpc]')

ax1.legend()

ax2.plot(rr, 100*(gammat_discrete-gammat_discrete)/gammat_discrete, 'k.-', label = 'discrete')
ax2.plot(rr, 100*(gammat_distribution-gammat_discrete)/gammat_discrete, 'rx-', label = 'distribution, no approx')
ax2.plot(rr, 100*(gammat_beta-gammat_discrete)/gammat_discrete, 'b--', label = 'beta, no approx')

ax2.set_ylabel('%')
ax2.set_xlabel('radius [Mpc]')

ax2.legend()


In [ ]:
kappa_discrete = np.zeros(rr.size)

for i in range(rr.size):
    kappa_discrete[i] = np.mean(clmm.theory.compute_convergence(rr[i], cluster_m, concentration, cluster_z, 
                                            source_catalog['z'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='discrete'))
    
kappa_distribution = clmm.theory.compute_convergence(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', beta_kwargs={'zmax':zsrc_max})
    
kappa_beta = clmm.theory.compute_convergence(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta', beta_kwargs={'zmax':zsrc_max})

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True)

ax1.loglog(rr, kappa_discrete, 'k.-', label = 'discrete')
ax1.loglog(rr, kappa_distribution, 'rx-', label = 'distribution, no approx')
ax1.loglog(rr, kappa_beta, 'b--', label = 'beta, no approx')

ax1.set_ylabel('$\\kappa$')
ax1.set_xlabel('radius [Mpc]')

ax1.legend()

ax2.plot(rr, 100*(kappa_discrete-kappa_discrete)/kappa_discrete, 'k.-', label = 'discrete')
ax2.plot(rr, 100*(kappa_distribution-kappa_discrete)/kappa_discrete, 'rx-', label = 'distribution, no approx')
ax2.plot(rr, 100*(kappa_beta-kappa_discrete)/kappa_discrete, 'b--', label = 'beta, no approx')

ax2.set_ylabel('%')
ax2.set_xlabel('radius [Mpc]')

ax2.legend()


In [ ]:
mu_discrete = np.zeros(rr.size)

for i in range(rr.size):
    mu_discrete[i] = np.mean(clmm.theory.compute_magnification(rr[i], cluster_m, concentration, cluster_z, 
                                            source_catalog['z'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='discrete'))
    
mu_distribution = clmm.theory.compute_magnification(rr, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', approx=None, beta_kwargs={'zmax':zsrc_max})
    
mu_beta_1 = clmm.theory.compute_magnification(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta',  approx="order1", beta_kwargs={'zmax':zsrc_max})

mu_beta_2 = clmm.theory.compute_magnification(rr, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta',  approx="order2", beta_kwargs={'zmax':zsrc_max})

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True)

ax1.loglog(rr, mu_discrete, 'k.-', label = 'discrete')
ax1.loglog(rr, mu_distribution, 'rs--', label = 'distribution, no approx')
ax1.loglog(rr, mu_beta_1, 'bx-', label = 'beta, order 1 approx')
ax1.loglog(rr, mu_beta_2, 'b--', label = 'beta, order 2 approx')

ax1.set_ylabel('$\\mu$')
ax1.set_xlabel('radius [Mpc]')

ax1.legend()

ax2.plot(rr, 100*(mu_discrete-mu_discrete)/mu_discrete, 'k.-', label = 'discrete')
ax2.plot(rr, 100*(mu_distribution-mu_discrete)/mu_discrete, 'rs--', label = 'distribution, no approx')
ax2.plot(rr, 100*(mu_beta_1-mu_discrete)/mu_discrete, 'bx-', label = 'beta, order 1 approx')
ax2.plot(rr, 100*(mu_beta_2-mu_discrete)/mu_discrete, 'b--', label = 'beta, order 2 approx')

ax2.set_ylabel('%')
ax2.set_xlabel('radius [Mpc]')

ax2.legend()

In [ ]:
mu_bias_discrete = np.zeros(rr.size)

alpha = 2.7 #arbitrary value for the slope of the number density

for i in range(rr.size):
    mu_bias_discrete[i] = np.mean(clmm.theory.compute_magnification_bias(rr[i], alpha, cluster_m, concentration, cluster_z, 
                                            source_catalog['z'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='discrete'))
    
mu_bias_distribution = clmm.theory.compute_magnification_bias(rr, alpha, cluster_m, concentration, cluster_z, 
                                            model_z_distrib_dict['func'], cosmo, delta_mdef=500,  massdef='critical', z_src_info='distribution', approx=None, beta_kwargs={'zmax':zsrc_max})
    
mu_bias_beta_1 = clmm.theory.compute_magnification_bias(rr, alpha, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta',  approx="order1", beta_kwargs={'zmax':zsrc_max})

mu_bias_beta_2 = clmm.theory.compute_magnification_bias(rr, alpha, cluster_m, concentration, cluster_z, 
                                            [beta_s_mean, beta_s_square_mean], cosmo, delta_mdef=500,  massdef='critical', z_src_info='beta',  approx="order2", beta_kwargs={'zmax':zsrc_max})

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True)

ax1.loglog(rr, mu_bias_discrete, 'k.-', label = 'discrete')
ax1.loglog(rr, mu_bias_distribution, 'rs--', label = 'distribution, no approx')
ax1.loglog(rr, mu_bias_beta_1, 'bx-', label = 'beta, order 1 approx')
ax1.loglog(rr, mu_bias_beta_2, 'b--', label = 'beta, order 2 approx')


ax1.set_ylabel('$\\delta_{\\mu} + 1$')
ax1.set_xlabel('radius [Mpc]')

ax1.legend()

ax2.plot(rr, 100*(mu_bias_discrete-mu_bias_discrete)/mu_bias_discrete, 'k.-', label = 'discrete')
ax2.plot(rr, 100*(mu_bias_distribution-mu_bias_discrete)/mu_bias_discrete, 'rs--', label = 'distribution, no approx')
ax2.plot(rr, 100*(mu_bias_beta_1-mu_bias_discrete)/mu_bias_discrete, 'bx-', label = 'beta, order 1 approx')
ax2.plot(rr, 100*(mu_bias_beta_2-mu_bias_discrete)/mu_bias_discrete, 'b--', label = 'beta, order 2 approx')

ax2.set_ylabel('%')
ax2.set_xlabel('radius [Mpc]')

ax2.legend()

**When making approximation, there may be large differences (~few 10% level) in the inner regions (compare to when using the exact redshift of the sources), especially for magnification and magnification bias. However, these approaches are very fast to compute. The user as to chose the appropriate method depending on the use case.**